In [ ]:
# notebook to generate census attributres in percentages 

In [1]:
import os 
import pandas as pd 
import glob 
import numpy as np   
import sys
sys.path.append('/Users/gracecolverd/New_dataset') 

from src.utils import * 

In [9]:
def create_simple_census_perc(df, code_col, val_col, attr):
    total_households  = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', aggfunc='sum') 
    h = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', columns = code_col, aggfunc='sum') 
    perc_df = h.join(total_households)
    
    mapping_dict = {key: value for key, value in zip(df[code_col], df[val_col])}

    for k ,v in mapping_dict.items():
        perc_df[f'{attr}_perc_{v}'] = perc_df[k] / perc_df['Observation']

    column_sum = perc_df[[f'{attr}_perc_{v}' for v in mapping_dict.values() ]].sum(axis=1)
    is_sum_equal_to_1 = np.isclose(column_sum, 1)
    if len (perc_df[~is_sum_equal_to_1]) != 0:
        raise ValueError('The conversion to percentages has failed.' ) 

    
    perc_df[[f'{attr}_perc_{v}' for v in mapping_dict.values() ]].reset_index().to_csv(f'/Users/gracecolverd/New_dataset/data/census_attrs/{attr}.csv', index = False)
    print('attr saved, len ', len(perc_df))
    

def create_complex_census_attr(df, code_col, val_col, code_col2, val_col2, attr):
    mapping_dict = {key: value for key, value in zip(df[code_col], df[val_col])}
    mapping_dict2 = {key: value for key, value in zip(df[code_col2], df[val_col2])}

    total_households = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', aggfunc='sum')
    h = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', columns = [code_col, code_col2], aggfunc='sum')
    perc_df = h.join(total_households)
    

    for k ,v in mapping_dict.items():
        for k2, v2 in mapping_dict2.items():
            perc_df[f'{attr}_perc_{v}_{v2}'] = perc_df[k, k2] / perc_df['Observation']
    
    column_sum = perc_df[[f'{attr}_perc_{v}_{v2}' for v in mapping_dict.values() for v2 in mapping_dict2.values() ]].sum(axis=1)
    is_sum_equal_to_1 = np.isclose(column_sum, 1)
    if len (perc_df[~is_sum_equal_to_1]) != 0:
        raise ValueError('The conversion to percentages has failed.' ) 

    perc_df[[f'{attr}_perc_{v}_{v2}' for v in mapping_dict.values() for v2 in mapping_dict2.values() ]].reset_index().to_csv(f'/Users/gracecolverd/New_dataset/data/census_attrs/{attr}.csv', index = False)
    print('attr saved, len ', len(perc_df))

In [3]:
lookup = pd.read_csv('/Volumes/T9/Data_downloads/lookups/pcs_to_oa_mapping_census2021/PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='latin1')

/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_48157/1841741371.py:1: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('/Volumes/T9/Data_downloads/lookups/pcs_to_oa_mapping_census2021/PCD_OA_LSOA_MSOA_LAD_MAY22_UK_LU.csv', encoding='latin1')


In [4]:
res_lk = join_pc_map_three_pc(res, 'Postcode',  lookup  )

NameError: name 'res' is not defined

In [25]:
res_lk_oa = res_lk.merge(df, left_on='oa11cd', right_on='Output Areas Code', how = 'left'     )

In [10]:
hh_size = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', columns = 'Household size (9 categories) Code', aggfunc='sum') 

In [6]:
# household size average attribute 
import pandas as pd
import numpy as np
 
# load household size census attr  - code is 0-8 for corresponding number of people (includes 8+ as 8) 
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/household_size/TS017-2021-3-filtered-2024-03-04T16_36_34Z.csv') 

total_households  = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', aggfunc='sum') 
hh_size = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', columns = 'Household size (9 categories) Code', aggfunc='sum') 
perc_df = hh_size.join(total_households)

perc_df['perc_1'] = perc_df[1] / perc_df['Observation']    
perc_df['perc_2'] = perc_df[2] / perc_df['Observation']
perc_df['perc_3'] = perc_df[3] / perc_df['Observation']
perc_df['perc_4'] = perc_df[4] / perc_df['Observation']
perc_df['perc_5'] = perc_df[5] / perc_df['Observation']
perc_df['perc_6'] = perc_df[6] / perc_df['Observation']
perc_df['perc_7'] = perc_df[7] / perc_df['Observation']
perc_df['perc_8'] = perc_df[8] / perc_df['Observation']

# Calculate the sum of the columns perc_1 to perc_8
column_sum = perc_df[['perc_1', 'perc_2', 'perc_3', 'perc_4', 'perc_5', 'perc_6', 'perc_7', 'perc_8']].sum(axis=1)

# Check if the sum is equal to 1 for each row
is_sum_equal_to_1 = np.isclose(column_sum, 1)

# Print the rows where the sum is not equal to 1
if len (perc_df[~is_sum_equal_to_1]) != 0:
    raise ValueError('The conversion to percentages has failed.' ) 



def calculate_average_household_size(row):
    """
    Calculate the weighted average household size for a given row.
    
    Parameters:
    - row: pandas Series, a row from the DataFrame containing percentages.
    
    Returns:
    - weighted_average: float, the weighted average household size for the row.
    """
    household_sizes = range(1, 9)  # Assuming household sizes from 1 to 8
    total_weighted_size = 0
    total_sum = 0
    
    for size in household_sizes:
        num_obs = row[size]
        total_sum += num_obs * size 
    total_obs = row['Observation']
       
    weighted_average = total_sum / total_obs
    return weighted_average


df = pd.DataFrame(perc_df)

# Applying the calculation to each row
df['Average Household Size'] = df.apply(calculate_average_household_size, axis=1)

# To view the result
# print(df[['Output Area Codes', 'Weighted Average Household Size']])

df.reset_index()[['Output Areas Code' ,  'Average Household Size'   ]].to_csv('/Users/gracecolverd/New_dataset/data/census_attrs/household_size.csv', index = False)

In [7]:
# create percentage of households with diff deprivations per output area 
df['upd_indx'] = df['Household deprivation (6 categories) Code'].astype(int) - 1 

total_households  = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', aggfunc='sum') 

h_depr = pd.pivot_table(df, index = 'Output Areas Code', values = 'Observation', columns = 'upd_indx', aggfunc='sum') 
perc_df = h_depr.join(total_households)

if len(perc_df[perc_df[-9] >0]) > 0:
    raise ValueError('Some households present with value -9')    

perc_df['perc_households_w_deprivation0'] = perc_df[0] / perc_df['Observation']    
perc_df['perc_households_w_deprivation1'] = perc_df[1] / perc_df['Observation']    
perc_df['perc_households_w_deprivation2'] = perc_df[2] / perc_df['Observation']
perc_df['perc_households_w_deprivation3'] = perc_df[3] / perc_df['Observation']
perc_df['perc_households_w_deprivation4'] = perc_df[4] / perc_df['Observation']


# Calculate the sum of the columns perc_1 to perc_8
column_sum = perc_df[['perc_households_w_deprivation0', 'perc_households_w_deprivation1', 'perc_households_w_deprivation2', 'perc_households_w_deprivation3', 'perc_households_w_deprivation4']].sum(axis=1)

# Check if the sum is equal to 1 for each row
is_sum_equal_to_1 = np.isclose(column_sum, 1)

# Print the rows where the sum is not equal to 1
if len (perc_df[~is_sum_equal_to_1]) != 0:
    raise ValueError('The conversion to percentages has failed.' ) 

perc_df[['perc_households_w_deprivation0', 'perc_households_w_deprivation1', 'perc_households_w_deprivation2', 'perc_households_w_deprivation3', 'perc_households_w_deprivation4']].reset_index().to_csv('/Users/gracecolverd/New_dataset/data/census_attrs/household_deprivation.csv', index = False)



KeyError: 'Household deprivation (6 categories) Code'

In [10]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/occupation/TS063-2021-5-filtered-2024-03-04T15_38_25Z.csv')
code_col = 'Occupation (current) (10 categories) Code'
val_col = 'Occupation (current) (10 categories)'
attr = 'occupation'
create_simple_census_perc(df, code_col, val_col, attr)

attr saved, len  188880


In [11]:
df= pd.read_csv('/Volumes/T9/Data_downloads/census_2021/economic_activity/TS066-2021-6-filtered-2024-03-04T15_29_15Z.csv') 
code_col = 'Economic activity status (20 categories) Code'
val_col = 'Economic activity status (20 categories)'
attr = 'economic_activity'
create_simple_census_perc(df, code_col, val_col, attr)  

attr saved, len  188880


In [12]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/household_size/TS017-2021-3-filtered-2024-03-04T16_36_34Z.csv')
df.head()
create_simple_census_perc(df,'Household size (9 categories) Code', 'Household size (9 categories)', 'household_siz_perc' )

attr saved, len  188880


In [13]:
df=pd.read_csv('/Volumes/T9/Data_downloads/census_2021/ethnic_group/TS021-2021-3-filtered-2024-03-05T10_06_33Z.csv')
df.head()  
create_simple_census_perc(df, 'Ethnic group (20 categories) Code', 'Ethnic group (20 categories)', 'ethnic_group')

attr saved, len  188880


In [14]:
df= pd.read_csv('/Volumes/T9/Data_downloads/census_2021/sex_by_age/RM121-2021-1-filtered-2024-03-05T10_10_26Z.csv') 
df.head()
create_simple_census_perc(df, 'Sex (2 categories) Code', 'Sex (2 categories)', 'sex')

attr saved, len  188880


In [61]:
df= pd.read_csv('/Volumes/T9/Data_downloads/census_2021/sex_by_age/RM121-2021-1-filtered-2024-03-05T10_10_26Z.csv') 
df.head()
create_simple_census_perc(df, 'Age (23 categories) Code', 'Age (23 categories)', 'age')

attr saved, len  188880


In [15]:
# household comp by bedroom 
import pandas as pd 
import numpy as np 

df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/household_bedroom_number/RM059-2021-3-filtered-2024-03-04T15_28_43Z.csv') 
df.head() 

create_complex_census_attr(df, 'Household composition (6 categories) Code', 'Household composition (6 categories)', 'Number of Bedrooms (5 categories) Code', 'Number of Bedrooms (5 categories)',  'household_comp_by_bedroom' )

/var/folders/n5/_2d3c5sd5j3810327mknn0sh0000gn/T/ipykernel_48157/1202531282.py:27: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  perc_df = h.join(total_households)


attr saved, len  188880


In [16]:
df= pd.read_csv('/Volumes/T9/Data_downloads/census_2021/sex_by_age/RM121-2021-1-filtered-2024-03-05T10_10_26Z.csv') 
df.head()
create_complex_census_attr(df, 'Sex (2 categories) Code', 'Sex (2 categories)', 'Age (23 categories) Code', 'Age (23 categories)',  'sex_by_age' )

In [ ]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/occupancy_rating/TS052-2021-5-filtered-2024-04-18T10_01_16Z.csv')
df.head()

create_simple_census_perc(df, 'Occupancy rating for bedrooms (6 categories) Code', 'Occupancy rating for bedrooms (6 categories)', 'occupancy_rating')

attr saved, len  188880


In [ ]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/central_heating/TS046-2021-4-filtered-2024-04-18T10_02_28Z.csv')
df.head()
create_simple_census_perc(df, 'Type of central heating in household (13 categories) Code', 'Type of central heating in household (13 categories)', 'central_heating')

attr saved, len  188880


In [ ]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/highest_qualification/TS067-2021-3-filtered-2024-04-18T09_58_11Z.csv')
df.head()
create_simple_census_perc(df, 'Highest level of qualification (8 categories) Code', 'Highest level of qualification (8 categories)', 'highest_qual')

attr saved, len  188880


In [ ]:
df = pd.read_csv('/Volumes/T9/Data_downloads/census_2021/tenure/TS054-2021-4-filtered-2024-04-18T10_19_46Z.csv')
df.head()
create_simple_census_perc(df, 'Tenure of household (9 categories) Code', 'Tenure of household (9 categories)', 'tenure')

attr saved, len  188880


In [ ]:
df=pd.read_csv('/Volumes/T9/Data_downloads/census_2021/level_SE/TS062-2021-5-filtered-2024-04-18T10_23_30Z.csv')
df.head()
create_simple_census_perc(df, 'National Statistics Socio-economic Classification (NS-SeC) (10 categories) Code', 'National Statistics Socio-economic Classification (NS-SeC) (10 categories)', 'tenure')

attr saved, len  188880


In [38]:
average = perc_df.iloc[:, 0:9].mean(axis=1)


In [39]:
average

Output Areas Code
E00000001    10.444444
E00000003    12.111111
E00000005     7.000000
E00000007     9.666667
E00000010    13.888889
               ...    
W00010693     6.444444
W00010694    17.666667
W00010695     8.222222
W00010696    10.777778
W00010697    13.777778
Length: 188880, dtype: float64